## Module 2 Homework

In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the [Colab](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/02-dataframe-analysis/Module2_Colab_Working_with_the_data.ipynb) covered at the livestream to re-use the code snippets.

---

In [1]:
!pip install yfinance

In [1]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

import requests
import re

### Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happenned of Fridays?**

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/
Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).
Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given).
You may be inspired by the function `extract_numbers()` in [Code Snippet 4], or you can write your own function to "parse" a string.
Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all fillings during 2023, which happened on Fridays (`Date.dt.dayofweek()==4`). You should see 32 records in total, 24 of it is not null.

(additional: you can read about [S-1 IPO filing](https://www.dfinsolutions.com/knowledge-hub/thought-leadership/knowledge-resources/what-s-1-ipo-filing) to understand the context)

---

In [5]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [6]:
ipos = ipo_dfs[0]
ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     325 non-null    object
 1   Symbol          325 non-null    object
 2   Company Name    325 non-null    object
 3   Price Range     325 non-null    object
 4   Shares Offered  325 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


In [8]:
# convert 'Filing Date' to datetime
ipos['Filing Date'] = pd.to_datetime(ipos['Filing Date'], format='%b %d, %Y')

In [9]:
# convert 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs)
ipos['Shares Offered'] = pd.to_numeric(ipos['Shares Offered'], errors='coerce')

In [10]:
ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     325 non-null    datetime64[ns]
 1   Symbol          325 non-null    object        
 2   Company Name    325 non-null    object        
 3   Price Range     325 non-null    object        
 4   Shares Offered  252 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 12.8+ KB


In [11]:
def extract_numbers(input_string):
    '''function to extract the 'Price Range' numbers and do the average calculation'''
    prices = re.findall(r'\d+\.\d+', input_string)
    if len(prices) == 0:
        return [np.nan]
    elif len(prices) == 1:
        return float(prices[0])
    else:
        return np.mean([float(price) for price in prices])

# Ejemplos
examples = ["$8.00 - $10.00", "$10.00"]
for example in examples:
    res = extract_numbers(example)
    print(f"{example} ==> {res}")

$8.00 - $10.00 ==> 9.0
$10.00 ==> 10.0


In [12]:
# define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified,
# to the price (if only one number is provided), or to the average of 2 prices (if a range is given)

ipos['Avg_price'] = ipos['Price Range'].apply(lambda x:extract_numbers(x))

In [13]:
# define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price"
# (when both columns are defined; otherwise, it's NaN)


def calculate_shares_offered_value(row):
    '''function to calculate Shares_offered_value'''
    if not pd.isnull(row['Shares Offered']) and not pd.isnull(row['Avg_price']):
        return row['Shares Offered'] * row['Avg_price']
    else:
        return np.nan

# applies the function along the rows of the DataFrame
ipos['Shares_offered_value'] = ipos.apply(lambda row: calculate_shares_offered_value(row), axis=1)

In [14]:
ipos.tail()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
320,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,9.0,13500000.0
321,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,10.0,50000000.0
322,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,9.0,22500000.0
323,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,4.0,17200000.0
324,2018-12-27,FBOX,Fit Boxx Holdings Limited,$5.00 - $6.50,2000000.0,5.75,11500000.0


In [15]:
# Filter records for 2023 and Fridays
ipos_2023_fridays = ipos[(ipos['Filing Date'].dt.year == 2023) & (ipos['Filing Date'].dt.dayofweek == 4)]

In [16]:
ipos_2023_fridays.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 50 to 165
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           32 non-null     datetime64[ns]
 1   Symbol                32 non-null     object        
 2   Company Name          32 non-null     object        
 3   Price Range           32 non-null     object        
 4   Shares Offered        25 non-null     float64       
 5   Avg_price             32 non-null     object        
 6   Shares_offered_value  25 non-null     float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 2.0+ KB


In [17]:
# Add the value of the shares offered for these records
total_shares_offered_value = ipos_2023_fridays['Shares_offered_value'].sum()

# Convert the sum to millions of dollars and round to the nearest integer
total_shares_offered_value_millions = round(total_shares_offered_value / 1_000_000)

print("Total sum in $m for all filings during 2023 that happened on Fridays:", total_shares_offered_value_millions)


Total sum in $m for all filings during 2023 that happened on Fridays: 286


### Question 2:  IPOs "Fixed days hold" strategy


**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**


Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/).
Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks.
Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day).
Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:
* You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
* You can use the `DataFrame.describe()` function to get mean, min, max, 25-50-75% quantiles.


Addtional:
* You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
* What's your recommendation: Do you suggest pursuing this strategy for an optimal X?


---

In [18]:
# get all OHLCV daily prices for all stocks

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [19]:
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [20]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [21]:
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      64 non-null     object
 1   Symbol        64 non-null     object
 2   Company Name  64 non-null     object
 3   IPO Price     64 non-null     object
 4   Current       64 non-null     object
 5   Return        64 non-null     object
dtypes: object(6)
memory usage: 3.1+ KB


In [22]:
# "stacking dataframes"
# pd.concat() is used to concatenate the DataFrames vertically.
# The ignore_index=True parameter ensures that the resulting DataFrame has a new index,
# ignoring the original indices of the input DataFrames.
# The stacked_df now contains the concatenated DataFrame.
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

In [23]:
stacked_ipos_df.head(1)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.65,19.38%


In [24]:
# Need to convert everything to a proper type (date, str, int, float, etc.)
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      218 non-null    object
 1   Symbol        218 non-null    object
 2   Company Name  218 non-null    object
 3   IPO Price     218 non-null    object
 4   Current       218 non-null    object
 5   Return        218 non-null    object
dtypes: object(6)
memory usage: 10.3+ KB


In [25]:
# convert to datetime
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format='%b %d, %Y')

In [26]:
# Problem --> not always the columns are filled
missing_prices_df = stacked_ipos_df[stacked_ipos_df['IPO Price'].astype(str).str.find('-') >= 0]
missing_prices_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return


In [27]:
# it has some missing values --> use defensive errors='coerce' (if don't have time to crack into the data errors)
#     : pd.to_numeric() function call, which will convert problematic values to NaN.
#     otherwise you'll get a ValueError: Unable to parse string "-" at position 9
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'].str.replace('$', ''), errors='coerce')
# not sure why, but need to call it again to transform 'object' to 'float64'
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'])

In [28]:
# Convert "Current" column
stacked_ipos_df['Current'] = pd.to_numeric(stacked_ipos_df['Current'].str.replace('$', ''), errors='coerce')

# Convert 'Return' to numeric format (percentage)
stacked_ipos_df['Return'] = pd.to_numeric(stacked_ipos_df['Return'].str.replace('%', ''), errors='coerce') / 100

In [29]:
# Correctly applied transformations with 'defensive' techniques, but now not all are non-null
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      218 non-null    datetime64[ns]
 1   Symbol        218 non-null    object        
 2   Company Name  218 non-null    object        
 3   IPO Price     218 non-null    float64       
 4   Current       218 non-null    float64       
 5   Return        217 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 10.3+ KB


In [30]:
# simple way of checking NULLs
# (you need to understand how vector operations work .isnull() and calls chaining .isnull().sum())
stacked_ipos_df.isnull().sum()

IPO Date        0
Symbol          0
Company Name    0
IPO Price       0
Current         0
Return          1
dtype: int64

In [31]:
# Do you want to leave the record or not?
stacked_ipos_df[stacked_ipos_df.Return.isnull()]

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
22,2024-03-28,UBXG,U-BX Technology Ltd.,5.0,4.88,NaN


In [32]:
# now you can operate with columns as a numeric type
stacked_ipos_df['IPO Price'].mean()

11.060229357798166

In [33]:
stacked_ipos_df.head(1)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2024-05-01,VIK,Viking Holdings Ltd.,24.0,28.65,0.1938


In [34]:
# Descriptive Analytics of a dataset
stacked_ipos_df.describe()

,IPO Date,IPO Price,Current,Return
count,218,218.000000,218.000000,217.000000
mean,2023-09-08 03:44:35.229357824,11.060229,11.163394,-0.209519
min,2023-01-13 00:00:00,2.500000,0.000000,-0.999600
25%,2023-04-20 00:00:00,4.000000,1.225000,-0.734000
50%,2023-09-14 12:00:00,8.000000,5.680000,-0.212500
75%,2024-01-24 18:00:00,13.750000,10.857500,0.059000
max,2024-05-01 00:00:00,92.000000,120.520000,2.597500
std,NaN,11.245650,17.399621,0.646715


In [35]:
# Filter an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers
specific_date = pd.to_datetime('2024-03-01')
filter_ipos = stacked_ipos_df[stacked_ipos_df['IPO Date'] < specific_date]

In [36]:
filter_ipos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185 entries, 33 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      185 non-null    datetime64[ns]
 1   Symbol        185 non-null    object        
 2   Company Name  185 non-null    object        
 3   IPO Price     185 non-null    float64       
 4   Current       185 non-null    float64       
 5   Return        185 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 10.1+ KB


In [37]:
# remove 'RYZB', as it is no longer available on Yahoo Finance.
filter_ipos = filter_ipos.drop(filter_ipos.loc[filter_ipos['Symbol'] == 'RYZB'].index)

In [38]:
# ERROR:yfinance:['PTHR']: Exception('%ticker%: No timezone found, symbol may be delisted')
# remove 'PTHR'
filter_ipos = filter_ipos.drop(filter_ipos.loc[filter_ipos['Symbol'] == 'PTHR'].index)

In [39]:
filter_ipos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 33 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      183 non-null    datetime64[ns]
 1   Symbol        183 non-null    object        
 2   Company Name  183 non-null    object        
 3   IPO Price     183 non-null    float64       
 4   Current       183 non-null    float64       
 5   Return        183 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 10.0+ KB


In [40]:
# adjust the symbol name
filter_ipos['Symbol'] = filter_ipos['Symbol'].replace('IBAC', 'IBACU')

In [41]:
# get prices from yf

tickers = filter_ipos['Symbol'].tolist()
prices_dict = {}
for ticker in tickers:
    try:
        prices_df = yf.download(tickers=ticker, period="max", interval="1d")
        prices_dict[ticker] = prices_df
    except Exception as e:
        print(f"Error getting data for {ticker}: {str(e)}")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [42]:
data = []

for ticker, prices_df in prices_dict.items():
    min_date = prices_df.index.min()
    first_close_price = prices_df.loc[min_date, 'Adj Close']

    # iterate over the next 30 days
    growth_data = {}
    for i in range(1, 31):
        try:
            # Find the next business day
            next_business_day = prices_df.index[prices_df.index > min_date][i]
            next_day_close_price = prices_df.loc[next_business_day, 'Adj Close']

            # Calculate growth
            growth_next_day = (next_day_close_price - first_close_price) / first_close_price
            #growth_next_day = first_close_price / next_day_close_price

            # Save growth
            growth_data[f'Growth_{i+1}_Days'] = growth_next_day

        except IndexError:
            # If there are not enough business days, set the growth to None
            growth_data[f'Growth_{i+1}_Days'] = None

    # Save all data for the ticker
    data.append({'Ticker': ticker, 'Min_Date': min_date, 'Adj Close': first_close_price, **growth_data})

# Save into the DF
df = pd.DataFrame(data)

# Show the df
print(df)


    Ticker   Min_Date  Adj Close  Growth_2_Days  Growth_3_Days  Growth_4_Days  \
0     SMXT 2024-02-27      8.000      -0.388750      -0.390000      -0.417500   
1     VHAI 2024-02-26      1.635       0.021407      -0.155963      -0.094801   
2     DYCQ 2024-04-12     10.100       0.001980       0.001980       0.001980   
3     CHRO 2024-02-16      4.800      -0.004167       0.000000      -0.097917   
4     UMAC 2024-02-14      3.030      -0.013201      -0.169967      -0.181518   
..     ...        ...        ...            ...            ...            ...   
178    QSG 2023-01-25     12.520       0.006390       0.000799       0.006390   
179   CVKD 2023-01-20      4.120      -0.223301      -0.296116      -0.310680   
180   SKWD 2023-01-13     19.100      -0.003665      -0.021466      -0.029843   
181   ISRL 2023-02-28     10.160      -0.000984      -0.001968       0.000984   
182   MGOL 2023-01-13      4.650      -0.204301      -0.154839      -0.260215   

     Growth_5_Days  Growth_

In [43]:
# Calculate quartiles for each growth column
quartile_025 = df.filter(like='Growth_').quantile(0.25)
quartile_050 = df.filter(like='Growth_').quantile(0.50)
quartile_075 = df.filter(like='Growth_').quantile(0.75)

# Find the column with the highest quartile
column_with_highest_quartile_025 = quartile_025.idxmax()
column_with_highest_quartile_050 = quartile_050.idxmax()
column_with_highest_quartile_075 = quartile_075.idxmax()

# Show the column with the highest quartile
print("The column with the highest 0.25 quartile is:", column_with_highest_quartile_025)
print("The column with the highest 0.50 quartile is:", column_with_highest_quartile_050)
print("The column with the highest 0.75 quartile is:", column_with_highest_quartile_075)


The column with the highest 0.25 quartile is: Growth_2_Days
The column with the highest 0.50 quartile is: Growth_2_Days
The column with the highest 0.75 quartile is: Growth_28_Days


### Question 3: Is Growth Concentrated in the Largest Stocks?

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**


Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks
for 10 full years of data (2014-01-01 to 2023-12-31):

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`
<br/>

Now let's add the top 12-22 stocks (as of end-April 2024):
<br/>

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS.

Calculate  `growth_7d` for every stock and every day.
Get the average daily `growth_7d` for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:
| Date   |      ticker_category      |  growth_7d |
|----------|:-------------:|------:|
| 2014-01-01 |  LARGE | 1.011684 |
| 2014-01-01 |   LARGEST   |   1.011797 |

On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

---


In [69]:
# Define the lists of stocks
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']
LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']
NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']
NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']
LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [77]:
ALL_TICKERS = LARGEST_STOCKS + LARGE_STOCKS

In [78]:
# LARGEST_STOCKS

stocks_df = pd.DataFrame()
for i,ticker in enumerate(ALL_TICKERS):
    print(i,ticker)

    # Work with stock prices
    historyPrices = yf.download(tickers=ticker, start="2013-12-01", end="2023-12-31")

    # generate features for historical prices, and what we want to predict
    historyPrices['Ticker'] = ticker
    historyPrices['Year']= historyPrices.index.year
    historyPrices['Month'] = historyPrices.index.month
    historyPrices['Weekday'] = historyPrices.index.weekday
    historyPrices['Date'] = historyPrices.index.date

    # historical returns
    historyPrices['growth_7d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(-7)


    # sleep 1 sec between downloads - not to overload the API server
    time.sleep(1)

    if stocks_df.empty:
        stocks_df = historyPrices
    else:
        stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

# the data from days before is downloaded to calculate the growth and then it is deleted
stocks_df = stocks_df[stocks_df['Year'] > 2013]

print(stocks_df.head())



[*********************100%%**********************]  1 of 1 completed

0 MSFT



[*********************100%%**********************]  1 of 1 completed

1 AAPL


2 GOOG


...

65 KOTAKBANK.NS


[*********************100%%**********************]  1 of 1 completed


66 TITAN.NS


[*********************100%%**********************]  1 of 1 completed


         Open       High        Low      Close  Adj Close    Volume Ticker  \
21  37.349998  37.400002  37.099998  37.160000  31.233065  30632200   MSFT   
22  37.200001  37.220001  36.599998  36.910000  31.022928  31134800   MSFT   
23  36.849998  36.889999  36.110001  36.130001  30.367340  43603700   MSFT   
24  36.330002  36.490002  36.209999  36.410000  30.602684  35802800   MSFT   
25  36.000000  36.139999  35.580002  35.759998  30.056356  59971700   MSFT   

    Year  Month  Weekday        Date  growth_7d  
21  2014      1        3  2014-01-02   1.062321  
22  2014      1        4  2014-01-03   1.031582  
23  2014      1        0  2014-01-06   0.982862  
24  2014      1        1  2014-01-07   0.986988  
25  2014      1        2  2014-01-08   0.982958  


In [79]:
def get_ticker_type(ticker:str, largest_stocks, large_stocks):
  if ticker in largest_stocks:
    return 'LARGEST'
  elif ticker in large_stocks:
    return 'LARGE'
  else:
    return 'ERROR'

In [80]:
stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, LARGEST_STOCKS, LARGE_STOCKS))

In [91]:
print(stocks_df.head())

         Open       High        Low      Close  Adj Close    Volume Ticker  \
21  37.349998  37.400002  37.099998  37.160000  31.233065  30632200   MSFT   
22  37.200001  37.220001  36.599998  36.910000  31.022928  31134800   MSFT   
23  36.849998  36.889999  36.110001  36.130001  30.367340  43603700   MSFT   
24  36.330002  36.490002  36.209999  36.410000  30.602684  35802800   MSFT   
25  36.000000  36.139999  35.580002  35.759998  30.056356  59971700   MSFT   

    Year  Month  Weekday        Date  growth_7d ticker_type  
21  2014      1        3  2014-01-02   1.062321     LARGEST  
22  2014      1        4  2014-01-03   1.031582     LARGEST  
23  2014      1        0  2014-01-06   0.982862     LARGEST  
24  2014      1        1  2014-01-07   0.986988     LARGEST  
25  2014      1        2  2014-01-08   0.982958     LARGEST  


In [90]:
# average growth
stocks_df.groupby(by=['Date', 'ticker_type']).growth_7d.mean()

Date        ticker_type
2014-01-01  LARGE          1.026906
            LARGEST        1.036741
2014-01-02  LARGE          1.002888
            LARGEST        1.008916
2014-01-03  LARGE          0.992526
                             ...   
2023-12-27  LARGEST             NaN
2023-12-28  LARGE               NaN
            LARGEST             NaN
2023-12-29  LARGE               NaN
            LARGEST             NaN
Name: growth_7d, Length: 5190, dtype: float64

In [115]:
# pivot table to compare Large vs Largest
table = stocks_df.pivot_table(values='growth_7d', index='Date', columns='ticker_type', aggfunc='mean').reset_index()

In [116]:
table

ticker_type,Date,LARGE,LARGEST
0,2014-01-01,1.026906,1.036741
1,2014-01-02,1.002888,1.008916
2,2014-01-03,0.992526,1.002768
3,2014-01-06,0.985549,0.989610
4,2014-01-07,0.987609,0.994233
...,...,...,...
2583,2023-12-13,0.998244,0.987801
2584,2023-12-14,0.994243,0.991000
2585,2023-12-15,0.998524,0.988205
2586,2023-12-18,0.999462,0.990726


In [117]:
table['Large_outperforms_Largest'] = table.LARGE > table.LARGEST
table.head()

ticker_type,Date,LARGE,LARGEST,Large_outperforms_Largest
0,2014-01-01,1.026906,1.036741,False
1,2014-01-02,1.002888,1.008916,False
2,2014-01-03,0.992526,1.002768,False
3,2014-01-06,0.985549,0.989610,False
4,2014-01-07,0.987609,0.994233,False


In [118]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2588 entries, 0 to 2587
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       2588 non-null   object 
 1   LARGE                      2588 non-null   float64
 2   LARGEST                    2588 non-null   float64
 3   Large_outperforms_Largest  2588 non-null   bool   
dtypes: bool(1), float64(2), object(1)
memory usage: 63.3+ KB


In [119]:
large_outperforms_largest_count = table['Large_outperforms_Largest'].sum()
large_outperforms_largest_count

1401

In [120]:
total_days = len(table)
total_days

2588

In [123]:
percentage_large_outperforms_largest = round(large_outperforms_largest_count / total_days * 100)
print(f"Percentage of days when Large Stocks outperform Largest Stocks: {percentage_large_outperforms_largest}%")

Percentage of days when Large Stocks outperform Largest Stocks: 54%


### Question 4: Trying Another Technical Indicators strategy

**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**


First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31).
If you encounter any difficulties running the Colab - you can download it using this [link](https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing).

Let's assume you've learned about the awesome **CCI indicator** ([Commodity Channel Index](https://www.investopedia.com/terms/c/commoditychannelindex.asp)), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (`Date.dt.dayofweek()==4`).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ ( closest integer value) over many operations in 10 years?

One operation calculations: if you invested  1000 and received 1010 in 5 days - you add 10 to gross profit, if you received 980 - add -20 to gross profit.
You need to sum these results over all trades (460 times in 10 years).

Additional:
  * Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
  * are you still profitable on those trades?


---

In [127]:
!gdown --id 1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb
To: /content/stocks_df_combined_trunc_2014_2023.parquet.brotli
100% 47.3M/47.3M [00:02<00:00, 19.0MB/s]


In [ ]:
# prompt: como paso ahora este archivo a un dataframe?

import pandas as pd
# import the DataFrame
stocks_df = pd.read_csv('stocks_df.csv')

# Truncate to the last full 10 years of data (2014-01-01 to 2023-12-31)
start_date = '2014-01-01'
end_date = '2023-12-31'
stocks_df = stocks_df[(stocks_df['Date'] >= start_date) & (stocks_df['Date'] <= end_date)]

# Select only Fridays
stocks_df = stocks_df[stocks_df['Date'].dt.dayofweek == 4]

# Filter for CCI > 200
cci_threshold = 200
stocks_df_filtered = stocks_df[stocks_df['CCI'] > cci_threshold]

# Calculate the gross profit for each trade
stocks_df_filtered['gross_profit'] = stocks_df_filtered['Adj Close'] * 1000 / stocks_df_filtered['Adj Close'].shift(-5) - 1000

# Sum the gross profit over all trades
total_gross_profit = stocks_df_filtered['gross_profit'].sum() / 1000

# Round to the closest integer value
total_gross_profit_rounded = round(total_gross_profit)

print(f"Total gross profit (in thousands of $): {total_gross_profit_rounded}")

# Additional: Approximate fees calculation
fees_calculator_url = "https://www.degiro.ie/fees/calculator"
product = "Shares, USA and Canada"
amount_per_transaction = 1000
transactions_per_year = 460

# Use a web scraping library like Selenium or BeautifulSoup to extract the fees
# from the calculator website based on the provided information

# For simplicity, assume the fees are 0.1% per transaction
fees_per_transaction = 0.001 * amount_per_transaction

# Calculate the total fees for all trades
total_fees = fees_per_transaction * transactions_per_year * 10

# Subtract the fees from the gross profit
net_profit = total_gross_profit - total_fees

print(f"Net profit (in thousands of $): {net_profit}")



In [129]:
# Load the data from Google Drive
url = 'https://drive.google.com/uc?id=1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb'
#df = pd.read_csv(url)
df = pd.read_parquet(url, engine='pyarrow')

In [134]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume', 'Ticker',
       'Year', 'Month', 'Weekday',
       ...
       'growth_brent_oil_7d', 'growth_brent_oil_30d', 'growth_brent_oil_90d',
       'growth_brent_oil_365d', 'growth_btc_usd_1d', 'growth_btc_usd_3d',
       'growth_btc_usd_7d', 'growth_btc_usd_30d', 'growth_btc_usd_90d',
       'growth_btc_usd_365d'],
      dtype='object', length=202)

In [153]:
df.head()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
7011,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
7012,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
7013,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
7014,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
7015,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
# Truncate the data to the last full 10 years (2014-01-01 to 2023-12-31)
df = df[df['Date'] >= '2014-01-01']
df = df[df['Date'] <= '2023-12-31']

In [137]:
# Select only the records when CCI is >200 and Date is Friday
df_filtered = df[df['cci'] > 200]
df_filtered = df_filtered[df_filtered['Date'].dt.dayofweek == 4]

In [178]:
# How many trades were there?
print(len(df_filtered))

460


In [173]:
# Calculate the gross profit for each operation on thousands
df_filtered['gross_profit'] =  (df_filtered['Adj Close_x']/ df_filtered['Adj Close_x'].shift(-5) )-1

In [165]:
# Calculate the gross profit for each trade
# df_filtered['gross_profit2'] = df_filtered['Adj Close_x'] * 1000 / df_filtered['Adj Close_x'].shift(-5) - 1000

In [176]:
# Sum the gross profit over all trades
total_gross_profit = df_filtered['gross_profit'].sum()
total_gross_profit

1837.6308063328588

In [177]:
# Convert the total gross profit to thousands of dollars and round to the nearest integer
total_gross_profit_thousands = round(total_gross_profit / 1000)

print("Total gross profit (in thousands of $):", total_gross_profit_thousands)

Total gross profit (in thousands of $): 2


Additional:

Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")

In [181]:
total_fees = 2210.50

In [182]:
# Convert the total fees to thousands of dollars and round to the nearest integer
total_fees_thousands = round(total_fees / 1000)
print("Total fees (in thousands of $):", total_fees_thousands)

Total fees (in thousands of $): 2


In [186]:
# Calculate the net profit
net_profit = total_gross_profit - total_fees
# Convert the net profit to thousands of dollars and round to the nearest integer
net_profit_thousands = round(net_profit / 1000)
print("Total net profit (in thousands of $):", net_profit_thousands)
print("Total net profit ($):", net_profit)

Total net profit (in thousands of $): 0
Total net profit ($): -372.8691936671412


### [EXPLORATORY] Question 5: Finding Your Strategy for IPOs

You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?

---

For the same IPOs selected above, I am going to calculate a strategy where I buy 1000 at the time the stock opens for the first time and sell it when the price closes on the first day.

In [66]:
data_1 = []

for ticker, prices_df in prices_dict.items():
    min_date = prices_df.index.min()
    first_open_price = prices_df.loc[min_date, 'Open']
    first_close_price = prices_df.loc[min_date, 'Adj Close']

    # iterate over the next 30 days
    growth_data = (first_close_price - first_open_price) / first_open_price


    # Save all data for the ticker
    data_1.append({'Ticker': ticker, 'Min_Date': min_date, 'Open': first_open_price,'Adj Close': first_close_price, 'Growth':growth_data})

# Save into the DF
df_new = pd.DataFrame(data_1)

# Show the df
print(df_new)

    Ticker   Min_Date   Open  Adj Close    Growth
0     SMXT 2024-02-27   3.50      8.000  1.285714
1     VHAI 2024-02-26   1.39      1.635  0.176259
2     DYCQ 2024-04-12  10.09     10.100  0.000991
3     CHRO 2024-02-16   6.00      4.800 -0.200000
4     UMAC 2024-02-14   4.00      3.030 -0.242500
..     ...        ...    ...        ...       ...
178    QSG 2023-01-25  13.70     12.520 -0.086131
179   CVKD 2023-01-20   5.71      4.120 -0.278459
180   SKWD 2023-01-13  18.90     19.100  0.010582
181   ISRL 2023-02-28  10.20     10.160 -0.003922
182   MGOL 2023-01-13   8.60      4.650 -0.459302

[183 rows x 5 columns]


In [61]:
total_gross_profit = df_new.Growth.sum()
total_gross_profit

6.98253712814299

In [62]:
#profit in 1000
total_gross_profit = df_new.Growth.sum()*1000
total_gross_profit

6982.53712814299

 From this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "183")
 € 880,90

 Assuming it is only 1 year

In [63]:
total_fees = 880.90

In [65]:
# Calculate the net profit
net_profit = total_gross_profit - total_fees
print("Total net profit ($):", round(net_profit,2))

Total net profit ($): 6101.64


## Submitting the solutions

Form for submitting: https://courses.datatalks.club/sma-zoomcamp-2024/homework/hw02

---